In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DPR"
outPath = "Results"
foldName = "folds.pickle"

shuffle = True
seed = None

input_data_folder = "Data_from_Asim"
file = "Protein_DistancePair[100, 0, 0, 0]-st-simplesequence.csv"

monitor = "val_accuracy"

In [2]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, matthews_corrcoef

import math

In [3]:
# for root, dirs, files in os.walk(input_data_folder):
#     for file in files:
#         print(file)

In [4]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [6]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred).astype(int)
    return y_pred

# Prepare Training and Independent data

In [7]:
input_data_file = os.path.join(input_data_folder, file)

data = pd.read_csv(input_data_file, sep=',', header=0)

train_data = data[data['set'] == 'train'].drop('set', axis=1)
independent_data = data[data['set'] == 'test'].drop('set', axis=1)

train_labels = np.array(train_data['labels'])
train_labels = train_labels.reshape((train_labels.shape[0], 1))

train_features = np.array(train_data.drop('labels', axis=1))

indpe_labels = np.array(independent_data['labels'])
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

indpe_features = np.array(independent_data.drop('labels', axis=1))

##################################################################################
##### extract data from the current fasta file
##################################################################################

print("\n======================================================================")
print("\nFile:", file)
print("Training Positive:", np.sum(train_labels))
print("Training Negative:", train_labels.shape[0] - np.sum(train_labels))
print("Independent Positive:", np.sum(indpe_labels))
print("Independent Negative:", indpe_labels.shape[0] - np.sum(indpe_labels))
print("Feature size:", train_features[0].shape)

##################################################################################
##### Generate Folds from dataset, and store to file
##################################################################################

## Generate the k-fold dataset
folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))



File: Protein_DistancePair[100, 0, 0, 0]-st-simplesequence.csv
Training Positive: 1191
Training Negative: 1191
Independent Positive: 203
Independent Negative: 1022
Feature size: (3629,)


# Model

In [8]:
epochs = 100
batch_size = 16

##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_Classifier(input_vec_shape,
                    dense_decode_units = 32, ## Dense layer parameters
                    prob = 0.5, learn_rate = 0.0005, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(input1)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(int(dense_decode_units/2), 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(y)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss)

    return model

In [9]:
DLNN_Classifier((3629,)).summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3629)]            0         
                                                                 
 dense (Dense)               (None, 32)                116160    
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 116,705
Trainable params: 116,705
Non-trainable

# Training

In [10]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

##################################################################################
##### TRAIN and PREDICT for every Fold, using models
##################################################################################

# fold counter
i = 0

for fold in folds:

    # adding random shuffling of the dataset for training purpose
    randomized_index_arr = np.arange(fold["X_train"].shape[0])
    randomized_index_arr = np.random.permutation(randomized_index_arr)

    print("\nTrain/Test model on Fold #"+str(i)+".")

    input_size = fold["X_train"][0].shape
    ## Generate model using function
    model = DLNN_Classifier(input_vec_shape = input_size)

    model_file_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    ## Define the model callbacks for early stopping and saving the model. Then train model
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(model_file_path,
                                           monitor = monitor, verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    model.fit(x = fold["X_train"][randomized_index_arr], y = fold["y_train"][randomized_index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))

    model = tf.keras.models.load_model(model_file_path)

    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)

    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)

    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)

    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)

    i = i+1
    del model
    tf.keras.backend.clear_session()


Train/Test model on Fold #0.
Epoch 1/100
109/120 [==========================>...] - ETA: 0s - loss: 0.7468 - accuracy: 0.5573
Epoch 1: val_accuracy improved from -inf to 0.74843, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DPR\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 5ms/step - loss: 0.7450 - accuracy: 0.5580 - val_loss: 0.7177 - val_accuracy: 0.7484
Epoch 2/100
109/120 [==========================>...] - ETA: 0s - loss: 0.7028 - accuracy: 0.6588
Epoch 2: val_accuracy did not improve from 0.74843
120/120 [==============================] - 0s 4ms/step - loss: 0.7015 - accuracy: 0.6604 - val_loss: 0.6790 - val_accuracy: 0.7317
Epoch 3/100
112/120 [===========================>..] - ETA: 0s - loss: 0.6637 - accuracy: 0.7254
Epoch 3: val_accuracy improved from 0.74843 to 0.75262, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DPR\5fold\models\bestModel-fold0.hdf5
120/120 [========================

Epoch 26/100
109/120 [==========================>...] - ETA: 0s - loss: 0.2830 - accuracy: 0.9736
Epoch 26: val_accuracy did not improve from 0.81761
120/120 [==============================] - 0s 4ms/step - loss: 0.2827 - accuracy: 0.9711 - val_loss: 0.5891 - val_accuracy: 0.8071
Epoch 27/100
114/120 [===========================>..] - ETA: 0s - loss: 0.2848 - accuracy: 0.9677
Epoch 27: val_accuracy did not improve from 0.81761
120/120 [==============================] - 0s 4ms/step - loss: 0.2850 - accuracy: 0.9659 - val_loss: 0.5822 - val_accuracy: 0.8176
Epoch 28/100
119/120 [============================>.] - ETA: 0s - loss: 0.2870 - accuracy: 0.9653
Epoch 28: val_accuracy did not improve from 0.81761
120/120 [==============================] - 0s 4ms/step - loss: 0.2870 - accuracy: 0.9654 - val_loss: 0.5929 - val_accuracy: 0.8050
Epoch 29/100
115/120 [===========================>..] - ETA: 0s - loss: 0.2800 - accuracy: 0.9668
Epoch 29: val_accuracy did not improve from 0.81761
120/120

Epoch 55/100
106/120 [=========================>....] - ETA: 0s - loss: 0.2575 - accuracy: 0.9741
Epoch 55: val_accuracy did not improve from 0.82180
120/120 [==============================] - 0s 4ms/step - loss: 0.2597 - accuracy: 0.9743 - val_loss: 0.6384 - val_accuracy: 0.8092
Epoch 56/100
112/120 [===========================>..] - ETA: 0s - loss: 0.2510 - accuracy: 0.9799
Epoch 56: val_accuracy did not improve from 0.82180
120/120 [==============================] - 0s 4ms/step - loss: 0.2512 - accuracy: 0.9795 - val_loss: 0.6610 - val_accuracy: 0.8092
Epoch 57/100
114/120 [===========================>..] - ETA: 0s - loss: 0.2598 - accuracy: 0.9742
Epoch 57: val_accuracy did not improve from 0.82180
120/120 [==============================] - 0s 4ms/step - loss: 0.2587 - accuracy: 0.9748 - val_loss: 0.6627 - val_accuracy: 0.8071
Epoch 58/100
110/120 [==========================>...] - ETA: 0s - loss: 0.2591 - accuracy: 0.9767
Epoch 58: val_accuracy did not improve from 0.82180
120/120

Epoch 11/100
116/120 [============================>.] - ETA: 0s - loss: 0.4373 - accuracy: 0.8885
Epoch 11: val_accuracy improved from 0.80084 to 0.82180, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DPR\5fold\models\bestModel-fold1.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.4367 - accuracy: 0.8887 - val_loss: 0.5437 - val_accuracy: 0.8218
Epoch 12/100
107/120 [=========================>....] - ETA: 0s - loss: 0.4159 - accuracy: 0.9001
Epoch 12: val_accuracy did not improve from 0.82180
120/120 [==============================] - 0s 4ms/step - loss: 0.4146 - accuracy: 0.9013 - val_loss: 0.5486 - val_accuracy: 0.7945
Epoch 13/100
105/120 [=========================>....] - ETA: 0s - loss: 0.4029 - accuracy: 0.9161
Epoch 13: val_accuracy did not improve from 0.82180
120/120 [==============================] - 0s 4ms/step - loss: 0.4051 - accuracy: 0.9155 - val_loss: 0.5519 - val_accuracy: 0.7987
Epoch 14/100
120/120 [==================

Epoch 40/100
108/120 [==========================>...] - ETA: 0s - loss: 0.2719 - accuracy: 0.9688
Epoch 40: val_accuracy did not improve from 0.82180
120/120 [==============================] - 0s 4ms/step - loss: 0.2693 - accuracy: 0.9701 - val_loss: 0.6752 - val_accuracy: 0.7987
Epoch 41/100
106/120 [=========================>....] - ETA: 0s - loss: 0.2661 - accuracy: 0.9741
Epoch 41: val_accuracy did not improve from 0.82180
120/120 [==============================] - 0s 4ms/step - loss: 0.2723 - accuracy: 0.9701 - val_loss: 0.6751 - val_accuracy: 0.7925
Epoch 42/100
109/120 [==========================>...] - ETA: 0s - loss: 0.2727 - accuracy: 0.9656
Epoch 42: val_accuracy did not improve from 0.82180
120/120 [==============================] - 0s 4ms/step - loss: 0.2708 - accuracy: 0.9669 - val_loss: 0.6798 - val_accuracy: 0.8008
Epoch 43/100
117/120 [============================>.] - ETA: 0s - loss: 0.2697 - accuracy: 0.9728
Epoch 43: val_accuracy did not improve from 0.82180
120/120

...

120/120 [==============================] - 1s 4ms/step - loss: 0.4995 - accuracy: 0.8395 - val_loss: 0.5520 - val_accuracy: 0.8109
Epoch 9/100
109/120 [==========================>...] - ETA: 0s - loss: 0.4643 - accuracy: 0.8693
Epoch 9: val_accuracy did not improve from 0.81092
120/120 [==============================] - 0s 4ms/step - loss: 0.4631 - accuracy: 0.8683 - val_loss: 0.5462 - val_accuracy: 0.8025
Epoch 10/100
109/120 [==========================>...] - ETA: 0s - loss: 0.4404 - accuracy: 0.8899
Epoch 10: val_accuracy improved from 0.81092 to 0.81933, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DPR\5fold\models\bestModel-fold3.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.4507 - accuracy: 0.8851 - val_loss: 0.5398 - val_accuracy: 0.8193
Epoch 11/100
108/120 [==========================>...] - ETA: 0s - loss: 0.4243 - accuracy: 0.8981
Epoch 11: val_accuracy did not improve from 0.81933
120/120 [==============================] -

120/120 [==============================] - 0s 4ms/step - loss: 0.2753 - accuracy: 0.9701 - val_loss: 0.6629 - val_accuracy: 0.8214
Epoch 37/100
106/120 [=========================>....] - ETA: 0s - loss: 0.2801 - accuracy: 0.9682
Epoch 37: val_accuracy did not improve from 0.82983
120/120 [==============================] - 0s 4ms/step - loss: 0.2804 - accuracy: 0.9690 - val_loss: 0.6587 - val_accuracy: 0.8214
Epoch 38/100
116/120 [============================>.] - ETA: 0s - loss: 0.2654 - accuracy: 0.9731
Epoch 38: val_accuracy did not improve from 0.82983
120/120 [==============================] - 0s 4ms/step - loss: 0.2645 - accuracy: 0.9738 - val_loss: 0.6574 - val_accuracy: 0.8193
Epoch 39/100
108/120 [==========================>...] - ETA: 0s - loss: 0.2627 - accuracy: 0.9716
Epoch 39: val_accuracy did not improve from 0.82983
120/120 [==============================] - 0s 4ms/step - loss: 0.2668 - accuracy: 0.9701 - val_loss: 0.6655 - val_accuracy: 0.8235
Epoch 40/100
117/120 [====

Epoch 66/100
119/120 [============================>.] - ETA: 0s - loss: 0.2543 - accuracy: 0.9758
Epoch 66: val_accuracy did not improve from 0.82983
120/120 [==============================] - 0s 4ms/step - loss: 0.2543 - accuracy: 0.9759 - val_loss: 0.6904 - val_accuracy: 0.8214
Epoch 67/100
109/120 [==========================>...] - ETA: 0s - loss: 0.2551 - accuracy: 0.9765
Epoch 67: val_accuracy did not improve from 0.82983
120/120 [==============================] - 0s 4ms/step - loss: 0.2573 - accuracy: 0.9759 - val_loss: 0.7080 - val_accuracy: 0.8172
Epoch 68/100
108/120 [==========================>...] - ETA: 0s - loss: 0.2571 - accuracy: 0.9745
Epoch 68: val_accuracy did not improve from 0.82983
120/120 [==============================] - 0s 4ms/step - loss: 0.2581 - accuracy: 0.9732 - val_loss: 0.7173 - val_accuracy: 0.8109
Epoch 69/100
110/120 [==========================>...] - ETA: 0s - loss: 0.2491 - accuracy: 0.9778
Epoch 69: val_accuracy did not improve from 0.82983
120/120

120/120 [==============================] - 0s 4ms/step - loss: 0.3203 - accuracy: 0.9580 - val_loss: 0.6056 - val_accuracy: 0.7920
Epoch 21/100
109/120 [==========================>...] - ETA: 0s - loss: 0.3022 - accuracy: 0.9662
Epoch 21: val_accuracy did not improve from 0.80462
120/120 [==============================] - 0s 4ms/step - loss: 0.3083 - accuracy: 0.9627 - val_loss: 0.6096 - val_accuracy: 0.8004
Epoch 22/100
114/120 [===========================>..] - ETA: 0s - loss: 0.3002 - accuracy: 0.9605
Epoch 22: val_accuracy did not improve from 0.80462
120/120 [==============================] - 0s 4ms/step - loss: 0.3007 - accuracy: 0.9607 - val_loss: 0.6126 - val_accuracy: 0.8046
Epoch 23/100
117/120 [============================>.] - ETA: 0s - loss: 0.2989 - accuracy: 0.9674
Epoch 23: val_accuracy did not improve from 0.80462
120/120 [==============================] - 0s 4ms/step - loss: 0.2994 - accuracy: 0.9675 - val_loss: 0.6239 - val_accuracy: 0.8004
Epoch 24/100
109/120 [====

117/120 [============================>.] - ETA: 0s - loss: 0.2546 - accuracy: 0.9733
Epoch 78: val_accuracy did not improve from 0.80882
120/120 [==============================] - 0s 4ms/step - loss: 0.2542 - accuracy: 0.9738 - val_loss: 0.7364 - val_accuracy: 0.7920
Epoch 79/100
120/120 [==============================] - ETA: 0s - loss: 0.2459 - accuracy: 0.9827
Epoch 79: val_accuracy did not improve from 0.80882
120/120 [==============================] - 0s 4ms/step - loss: 0.2459 - accuracy: 0.9827 - val_loss: 0.7441 - val_accuracy: 0.7962
Epoch 80/100
109/120 [==========================>...] - ETA: 0s - loss: 0.2522 - accuracy: 0.9771
Epoch 80: val_accuracy did not improve from 0.80882
120/120 [==============================] - 0s 4ms/step - loss: 0.2526 - accuracy: 0.9774 - val_loss: 0.7426 - val_accuracy: 0.7962
Epoch 81/100
113/120 [===========================>..] - ETA: 0s - loss: 0.2471 - accuracy: 0.9823
Epoch 81: val_accuracy did not improve from 0.80882
120/120 [===========

## k-fold Training evaluation

In [11]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.817377,0.815270,0.88234,0.821968,0.812753,0.636067
Train,0.979008,0.980636,0.99520,0.977333,0.980682,0.958030


In [14]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Train,0.991601,0.993671,"[0.0, 0.0010504201680672268, 0.118697478991596...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.999528, 0.99952805, 0.9948559, 0.99485224, ...",0.999751,0.989496,0.993704,0.983211
1,0,Test,0.821803,0.800781,"[0.0, 0.0041841004184100415, 0.075313807531380...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9989052, 0.9989052, 0.99469674, 0.9945716, ...",0.892708,0.857741,0.785714,0.645193
2,1,Train,0.952231,0.951782,"[0.0, 0.001049317943336831, 0.0367261280167890...","[0.0, 0.0, 0.0, 0.0010504201680672268, 0.00105...","[1.9725653, 0.9725653, 0.9255821, 0.9248224, 0...",0.985101,0.952781,0.951681,0.904462
3,1,Test,0.821803,0.822785,"[0.0, 0.004201680672268907, 0.1386554621848739...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[1.9545853, 0.9545854, 0.8967029, 0.895575, 0....",0.876701,0.819328,0.824268,0.643607
4,2,Train,0.970094,0.975584,"[0.0, 0.001049317943336831, 0.1280167890870933...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9936662, 0.99366623, 0.9564874, 0.95621395,...",0.992373,0.964323,0.975866,0.940252
5,2,Test,0.804622,0.837209,"[0.0, 0.004201680672268907, 0.1722689075630252...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.989124, 0.98912394, 0.9294979, 0.9292516, 0...",0.883280,0.756303,0.852941,0.612109
6,3,Train,0.990031,0.990546,"[0.0, 0.001049317943336831, 0.1605456453305351...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9993931, 0.9993931, 0.99036926, 0.99030846,...",0.999753,0.989507,0.990556,0.980063
7,3,Test,0.829832,0.805447,"[0.0, 0.004201680672268907, 0.1176470588235294...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9990675, 0.9990675, 0.9916814, 0.9906454, 0...",0.880217,0.869748,0.789916,0.661776
8,4,Train,0.991081,0.991597,"[0.0, 0.001049317943336831, 0.1143756558237145...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9981375, 0.9981375, 0.98620725, 0.9861667, ...",0.999023,0.990556,0.991605,0.982162
9,4,Test,0.808824,0.810127,"[0.0, 0.004201680672268907, 0.0504201680672268...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.996172, 0.9961719, 0.9883697, 0.9879759, 0....",0.878796,0.806723,0.810924,0.617653


# Independent data

In [15]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Independent Data performance
##################################################################################

print("\nIndependent evaluation for model.")

# adding random shuffling of the dataset for training purpose
randomized_index_arr = np.arange(train_features.shape[0])
randomized_index_arr = np.random.permutation(randomized_index_arr)

input_size = train_features[0].shape
## Generate model using function
model = DLNN_Classifier(input_vec_shape = input_size)

model_file_path = os.path.join(modelPath, "bestModel-full.hdf5")
## Define the model callbacks for early stopping and saving the model. Then train model
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(model_file_path,
                                       monitor = monitor, verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]
model.fit(x = train_features[randomized_index_arr], y = train_labels[randomized_index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))

model = tf.keras.models.load_model(model_file_path)

##################################################################################
##### Prediction and metrics for Train dataset
##################################################################################

y_pred = model.predict(train_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(train_labels, label_pred)
prec = precision_score(train_labels,label_pred)
mcc = matthews_corrcoef(train_labels, label_pred)

conf = confusion_matrix(train_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(train_labels, y_pred)
auc = roc_auc_score(train_labels, y_pred)

evaluations["Train_Test"].append("Train")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

del model
tf.keras.backend.clear_session()


Independent evaluation for model.
Epoch 1/100
149/149 [==============================] - ETA: 0s - loss: 0.7351 - accuracy: 0.5730
Epoch 1: val_accuracy improved from -inf to 0.64735, saving model to Results\NT_Site_PredNTS_Classification_DLNN_AsimEmbedding_DPR\5fold\models\bestModel-full.hdf5
149/149 [==============================] - 1s 6ms/step - loss: 0.7351 - accuracy: 0.5730 - val_loss: 0.7135 - val_accuracy: 0.6473
Epoch 2/100
142/149 [===========================>..] - ETA: 0s - loss: 0.6829 - accuracy: 0.6813
Epoch 2: val_accuracy did not improve from 0.64735
149/149 [==============================] - 1s 4ms/step - loss: 0.6830 - accuracy: 0.6793 - val_loss: 0.7220 - val_accuracy: 0.5584
Epoch 3/100
139/149 [==========================>...] - ETA: 0s - loss: 0.6329 - accuracy: 0.7347
Epoch 3: val_accuracy did not improve from 0.64735
149/149 [==============================] - 1s 4ms/step - loss: 0.6319 - accuracy: 0.7338 - val_loss: 0.7429 - val_accuracy: 0.5584
Epoch 4/100
144

138/149 [==========================>...] - ETA: 0s - loss: 0.2660 - accuracy: 0.9769
Epoch 58: val_accuracy did not improve from 0.67020
149/149 [==============================] - 1s 4ms/step - loss: 0.2669 - accuracy: 0.9761 - val_loss: 1.3399 - val_accuracy: 0.6498
Epoch 59/100
138/149 [==========================>...] - ETA: 0s - loss: 0.2683 - accuracy: 0.9715
Epoch 59: val_accuracy did not improve from 0.67020
149/149 [==============================] - 1s 4ms/step - loss: 0.2679 - accuracy: 0.9719 - val_loss: 1.4299 - val_accuracy: 0.6204
Epoch 60/100
148/149 [============================>.] - ETA: 0s - loss: 0.2676 - accuracy: 0.9747
Epoch 60: val_accuracy did not improve from 0.67020
149/149 [==============================] - 1s 5ms/step - loss: 0.2684 - accuracy: 0.9744 - val_loss: 1.3971 - val_accuracy: 0.6220
Epoch 61/100
145/149 [============================>.] - ETA: 0s - loss: 0.2719 - accuracy: 0.9703
Epoch 61: val_accuracy did not improve from 0.67020
149/149 [===========

In [16]:
evaluations_df = pd.DataFrame.from_dict(evaluations)
evaluations_df

,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,Train,0.919815,0.951264,"[0.0, 0.0008396305625524769, 0.165407220822837...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9712874, 0.97128737, 0.8802528, 0.88024867,...",0.973134,0.884971,0.954660,0.841677
1,Independent,0.670204,0.270023,"[0.0, 0.0049261083743842365, 0.009852216748768...","[0.0, 0.0, 0.0, 0.0009784735812133072, 0.00097...","[1.970679, 0.9706791, 0.9595812, 0.9594154, 0....",0.683645,0.581281,0.687867,0.208911
